In [1]:
import tensorflow as tf
import numpy as np
tf.enable_eager_execution()
import importlib
import os

from transliteration import data, train, model_one, script

In [2]:
importlib.reload(data)
batch_size = 128
train_dataset = data.make_dataset('../data/tfrecord/train.tfrecord',
                                  from_script='en',
                                  to_script='ja',
                                  batch_size=batch_size)
valid_dataset = data.make_dataset('../data/tfrecord/valid.tfrecord',
                                  from_script='en',
                                  to_script='ja',
                                  batch_size=batch_size)
test_dataset = data.make_dataset('../data/tfrecord/test.tfrecord',
                                 from_script='en',
                                 to_script='ja',
                                 batch_size=batch_size)

In [3]:
optimizer = tf.train.AdamOptimizer()


def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred)
    return tf.reduce_mean(loss_ * mask)

In [4]:
importlib.reload(model_one)
encoder_config = model_one.Config(lstm_size=60,
                                  embedding_size=60,
                                  attention_size=30,
                                  vocab_size=script.SCRIPTS['en'].vocab_size)
decoder_config = model_one.Config(lstm_size=60,
                                  embedding_size=30,
                                  attention_size=30,
                                  vocab_size=script.SCRIPTS['ja'].vocab_size)
encoder = model_one.Encoder(encoder_config)
decoder = model_one.Decoder(decoder_config)

In [5]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [6]:
importlib.reload(train)
best_val_loss = None
for e in range(200):
    loss = train.run_one_epoch(train_dataset,
                               True,
                               from_script='en',
                               to_script='ja',
                               encoder=encoder,
                               decoder=decoder,
                               optimizer=optimizer,
                               loss_function=loss_function)
    valid_loss = train.run_one_epoch(valid_dataset,
                                     False,
                                     from_script='en',
                                     to_script='ja',
                                     encoder=encoder,
                                     decoder=decoder,
                                     loss_function=loss_function)
    if best_val_loss is None or valid_loss < best_val_loss:
        best_val_loss = valid_loss
        checkpoint.save(file_prefix=checkpoint_prefix)
    print("Epoch {}: Train Loss {:.3f}, Valid Loss {:.3f}".format(e, loss, valid_loss))
    print(train.transliterate(input_strs=['derick'],
                              from_script='en',
                              to_script='ja',
                              encoder=encoder,
                              decoder=decoder))

Instructions for updating:
Colocations handled automatically by placer.


Epoch 0: Train Loss 20.510, Valid Loss 19.166
['リーー']


Epoch 1: Train Loss 18.329, Valid Loss 17.981
['アリンルル']


Epoch 2: Train Loss 16.788, Valid Loss 16.224
['カラントラン']


Epoch 3: Train Loss 15.123, Valid Loss 14.806
['ディリート']


...

Epoch 55: Train Loss 2.862, Valid Loss 5.526
['デリック']


Epoch 56: Train Loss 2.816, Valid Loss 5.471
['デリック']


Epoch 57: Train Loss 2.848, Valid Loss 5.729
['デリック']


Epoch 58: Train Loss 2.813, Valid Loss 5.516
['デリック']


KeyboardInterrupt: 

In [23]:
importlib.reload(train)
importlib.reload(data)
train.transliterate(input_strs=['bob'],
                    from_script='en',
                    to_script='ja',
                    encoder=encoder,
                    decoder=decoder)

['ボボボ']

In [42]:
data.intern_katakana_char('<end>')

98